In [ ]:
import sys
sys.path.append('../')
sys.path.append('/Users/sdbykov/work/my_packages/nway')

In [ ]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import search_around_r_data_frames, cross_match_data_frames
import pandas as pd
import numpy as np
import seaborn as sns
import nway_python_api as nway
set_mpl()
%matplotlib inline

In [ ]:
erosita = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl')

desi =  pd.read_pickle(data_path+'desi_lh.gz_pkl', compression = 'gzip')
desi_matched = cross_match_data_frames(erosita, desi, 'RA_fin', 'DEC_fin', 'ra', 'dec', 30, df_prefix='desi')

xmm_matched = pd.read_pickle(data_path+'xmm_cleared.pkl')

csc = pd.read_pickle(data_path+'csc_cleared.pkl')
csc_matched = cross_match_data_frames(erosita, csc, 'RA_fin', 'DEC_fin', 'ra', 'dec', 30)

ctps = pd.read_pickle(data_path+'validation_ctps_ero_desi_lh.pkl')
ctps.ID = ctps.ID.str.decode('utf-8')
ctps.desi_id_true_ctp = ctps.desi_id_true_ctp.str.decode('utf-8')


In [ ]:
ctps

In [ ]:
def plot_target(eroid, savefig = False):

    df_query = erosita.query("srcname_fin==@eroid")

    ero_ra, ero_dec, ero_err = df_query.iloc[0][['RA_fin', 'DEC_fin', 'pos_r98']]

    search_rad = ero_err*1.5
    fig, ax = plt.subplots(figsize=(10, 10))

    csc_around = csc_matched.query("srcname_fin	== @eroid & sep<@search_rad")
    if len(csc_around) == 0:
        pass
        #print('No CSC')
    else:
        for row in csc_around.itertuples():
            ra, dec, err = row.ra, row.dec, row.r_98_csc
            ra = 3600*(ra - ero_ra)
            dec = 3600*(dec - ero_dec)
            circle = plt.Circle((ra, dec), err, color='b', fill=False, label = 'CSC')
            ax.add_artist(circle)
            circle = plt.Circle((ra, dec), 1.43, color='b', fill=False,  linestyle='dashed', alpha = 0.4)
            ax.add_artist(circle)
            


    xmm_around = xmm_matched.query("srcname_fin	== @eroid & Separation<@search_rad")
    if len(xmm_around) == 0:
        #print('No XMM')
        pass
    else:
        for row in xmm_around.itertuples():
            ra, dec, err = row.SC_RA, row.SC_DEC, row.xmm_pos_r98
            ra = 3600*(ra - ero_ra)
            dec = 3600*(dec - ero_dec)
            circle = plt.Circle((ra, dec), err, color='c', fill=False, label = 'XMM')
            ax.add_artist(circle)
            circle = plt.Circle((ra, dec), 1.43, color='c', fill=False,  linestyle='dashed', alpha = 0.4)
            ax.add_artist(circle)



    desi_around = desi_matched.query("srcname_fin == @eroid & desi_sep<@search_rad")
    if len(desi_around) == 0:
        pass
        #print('No DESI')
    else:
        for row in desi_around.itertuples():
            ra, dec, err = row.desi_ra, row.desi_dec, 0.25
            ra = 3600*(ra - ero_ra)
            dec = 3600*(dec - ero_dec)
            circle = plt.Circle((ra, dec), err, color='g', fill=False, label = '_DESI')
            ax.add_artist(circle)
            ax.text(ra, dec, row.desi_desi_id.split('_')[-1], color='g', fontsize=12)

    circle = plt.Circle((0, 0), ero_err, color='r', fill=False, label = 'eRO')
    ax.add_artist(circle)

    axislim = 3*ero_err

    ax.set_xlim(-axislim, axislim)
    ax.set_ylim(-axislim, axislim)

    ax.set_xlabel('delta RA, arcsec')
    ax.set_ylabel('delta DEC, arcsec')
    #invert x axis 
    ax.invert_xaxis()
    ax.legend()

    try:
        true_ctps = ctps.query("ID==@eroid").iloc[0].desi_id_true_ctp
    except:
        true_ctps = 'unknown'

    title = eroid + '|'+  true_ctps# + '|' +'RA,DEC'+ str(ero_ra)+','+ str(ero_dec)
    ax.set_title(title)
    if savefig:
        plt.savefig('plots/'+eroid.replace(' ','')+'_'+true_ctps+'.png', dpi=300)


# plot_target('SRGe J103118.8+574434')
# plot_target('SRGe J103130.8+573932') #nice example!
# plot_target('SRGe J110721.3+590113')


In [ ]:
from tqdm import tqdm_notebook as tqdm
#for eroid in tqdm(ctps.query('desi_id_true_ctp!="hostless"').sample(10).ID):
#    print(eroid)
#    plot_target(eroid, savefig = True)

for eroid in tqdm(ctps.query('desi_id_true_ctp=="hostless"').sample(5).ID):
    print(eroid)
    plot_target(eroid, savefig = True)


In [ ]:
# from tqdm import tqdm_notebook as tqdm
# for eroid in tqdm(ctps.query('desi_id_true_ctp=="hostless"').ID):
#     plot_target(eroid, savefig=True)
#     plt.close('all')
